In [534]:
import pandas as pd
import numpy as np
import pyspark
import os
import shutil
import copy

In [535]:
#----------------------------------------------------------------
# GET RELATION ACTIVITIES CATEGORIES
#----------------------------------------------------------------
def getRelationActivitiesCategories(tuplesList, activitiesDict):
    size = len(tuplesList)
    relation = {}
    for i in range(0, size):
        activity = tuplesList[i][0]
        category = tuplesList[i][1]
        if category not in relation:
            relation[category] = [activitiesDict[activity]]
        else: 
            relation[category].append(activitiesDict[activity])
    return relation
#----------------------------------------------------------------
# GET HASH NAMES
#----------------------------------------------------------------
def gethHashedNames(namesList):
    names = {}
    cont = 0
    size = len(namesList)
    for i in range(0, size):
        if namesList[i] in names: continue
        names[namesList[i]] = cont
        cont += 1
    return names
#----------------------------------------------------------------
# HASH NAMES
#----------------------------------------------------------------
def hashNames(x, pos):
    names = {}
    cont = 0
    x = list(x)
    size = len(x)
    for i in range(0, size):
        if x[i][pos] in names:
            x[i][pos] = names[x[i][pos]]
            continue
        names[x[i][pos]] = cont
        cont += 1
        x[i][pos] = names[x[i][pos]]
    return x
#----------------------------------------------------------------
# TOOK TIMES TO SECONDS
#----------------------------------------------------------------
def TookTimesToMinutes(aTookTime):
    # '2018-04-21T16:22:49.000Z'
    aTookTime = aTookTime.split(aTookTime[10])[1]
    aTookTime = aTookTime.split(aTookTime[8])[0]
    #16:22:49
    aTookTime = aTookTime.split(':')
    aTookTime[0] = int(aTookTime[0])
    aTookTime[1] = int(aTookTime[1])
    aTookTime[2] = int(aTookTime[2])
    if (aTookTime[0] < 13):
        aTookTime[0] += 24
    aTookTime[0] = aTookTime[0]*60*60
    aTookTime[1] = aTookTime[1]*60
    aTookTime = aTookTime[0] + aTookTime[1] + aTookTime[2]
    return aTookTime
#----------------------------------------------------------------
# CHANGE TOOK TIMES
#----------------------------------------------------------------
def changeTookTimes(x):
    size = len(x[1])
    # name, [(w, t), (w, t), ....]
    for j in range(0, size): x[1][j][1] = TookTimesToMinutes(x[1][j][1])
    vec = []
    for j in range(0, size):
        if (j == 0): vec.append([x[1][j][0], 0])
        else:
            newTookTime = x[1][j][1] - x[1][0][1]
            # 14:00:00: 840 -> 0 -> vec[0]
            # 14:01:00: 900 -> 60 -> 
            # 14:02:00: 960 -> 120
            # 14:05:00:  -> 300
            vec.append([x[1][j][0], newTookTime])
    x[1] = vec
    return x
#----------------------------------------------------------------
# CHANGE FORMAT
#----------------------------------------------------------------
def changeFormat(x):
    size = len(x[1])
    for j in range(0, size): 
        x[1][j] = [x[0], x[1][j][0], x[1][j][1]]
    return x[1]
#----------------------------------------------------------------
# DELETE REPETITIONS
#----------------------------------------------------------------
def deleteRepetitions(x):
    repeated = {}
    indexToDelete = []
    size = len(x[1])
    for i in range(0, size):
        tookTime = x[1][i][1]
        if tookTime not in repeated:
            repeated[tookTime] = True
        else:
            indexToDelete.append(i)
    for i in indexToDelete: 
        x[1].pop(i)
    return x
#----------------------------------------------------------------
# SHOW ERRORS
#----------------------------------------------------------------
def showErrors(disney):
    disney = disney.map(lambda x: (x[2], (1, x[0])))
    disney = disney.reduceByKey(lambda x, y: (x[0]+y[0], x[1]))
    disney = disney.collect()
    return [x for x in disney if x[1][0] != 22]
#----------------------------------------------------------------
# REMOVE TOOK TIMES
#----------------------------------------------------------------
def removeTookTimes(disney, listTookTimes):
    return disney.filter(lambda x: x[2] not in listTookTimes)
#----------------------------------------------------------------
# GET TOOK TIMES MINUTES
#----------------------------------------------------------------
def getTookTimeMinutes(aTookTime):
    #'2018-04-21T13:14:57.000Z'
    aTookTime = aTookTime.split(aTookTime[10])[1]
    aTookTime = aTookTime.split(aTookTime[8])[0]
    #13:14:57
    aTookTime = aTookTime.split(':')
    hours = int(aTookTime[0])
    if (hours < 13):
        hours += 24
    minutes = int(aTookTime[1])
    return hours*60 + minutes
#----------------------------------------------------------------
# REMOVE TOOK TIMES EVERY GAP
#----------------------------------------------------------------
def removeTookTimesEveryGap(reg, gap):
    vec = []
    size = len(reg[1])
    vec.append(reg[1][0])
    T0 = getTookTimeMinutes(reg[1][0][1])
    for i in range(1, size):
        T1 = getTookTimeMinutes(reg[1][i][1])
        if (T1 - T0 >= gap):
            T0 = T1
            vec.append(reg[1][i])
    reg[1] = vec
    return reg
#----------------------------------------------------------------
# SHOW TOOK TIME HASH
#----------------------------------------------------------------
def showTookTimehash(tookTimes):
    size = len(tookTimes)
    for i in range(0, size):
        print i, ": ", tookTimes[i][1]

In [536]:
gap = 45
carpet = "modelo" + str(gap)
dataName = carpet + "/datos" + str(gap) + ".txt"
modelName = carpet + "/model" + str(gap) + ".txt"

#  Parse Disney

In [537]:
disneyRoot = "SetDeDatos/disney.csv"
disney = spark.read.load(disneyRoot, format="csv", header=True).rdd
disney = disney.map(lambda line: list([str(x) for x in line]))
disney = disney.map(lambda x: [x[1], int(x[2]), x[3]])
disney = disney.map(lambda x: [x[0], [x[1], x[2]]]).groupByKey()
disney = disney.map(lambda x: [x[0], list(x[1])])
disney = disney.map(lambda x: removeTookTimesEveryGap(x, gap))
tookTimes = disney.collect()[0][1]
disney = disney.map(lambda x: deleteRepetitions(x))
disney = disney.map(lambda x: changeTookTimes(x))
disney = disney.mapPartitions(lambda x: hashNames(x, 0))
disney = disney.flatMap(lambda x: changeFormat(x))
disney = disney.map(lambda x: [x[0], x[1]*60, x[2]])
print "Errors: ", showErrors(disney)
print "Tooktimes: "
print showTookTimehash(tookTimes)
print "Parsed Disney: "
print disney.take(5)
disney = disney.collect()

Errors:  []
Tooktimes: 
0 :  2018-04-21T13:14:57.000Z
1 :  2018-04-21T13:59:48.000Z
2 :  2018-04-21T14:44:48.000Z
3 :  2018-04-21T15:29:48.000Z
4 :  2018-04-21T16:14:49.000Z
5 :  2018-04-21T16:59:49.000Z
6 :  2018-04-21T17:44:49.000Z
7 :  2018-04-21T18:29:49.000Z
8 :  2018-04-21T19:14:49.000Z
9 :  2018-04-21T19:59:49.000Z
10 :  2018-04-21T20:44:50.000Z
11 :  2018-04-21T21:29:50.000Z
12 :  2018-04-21T22:14:50.000Z
13 :  2018-04-21T22:59:50.000Z
14 :  2018-04-21T23:44:50.000Z
15 :  2018-04-22T00:29:50.000Z
16 :  2018-04-22T01:14:51.000Z
None
Parsed Disney: 
[[0, 600, 0], [0, 600, 2691], [0, 900, 5391], [0, 1200, 8091], [0, 2100, 10792]]


#  Parse Description

In [538]:
descriptionRoot = "SetDeDatos/descripcion.csv"
description = spark.read.load(descriptionRoot, format="csv", header=True).rdd
description = description.map(lambda line: list([str(x) for x in line]))
#-----------------------------------------------------------------------------
categoriesDict = gethHashedNames(description.map(lambda x: x[1]).collect())
activitiesDict = gethHashedNames(description.map(lambda x: x[0]).collect())
tupleList = description.map(lambda x: (x[0], x[1])).collect()
relation = getRelationActivitiesCategories(tupleList, activitiesDict)
#-----------------------------------------------------------------------------
categoriesDict["ACCION"] = categoriesDict.pop("accion")
categoriesDict["BOTES"] = categoriesDict.pop("botes")
categoriesDict["FUTURO"] = categoriesDict.pop("futuro")
categoriesDict["MONTANA"] = categoriesDict.pop("monta??a")
categoriesDict["MUSICAL"] = categoriesDict.pop("musical")
categoriesDict["NINOS"] = categoriesDict.pop("ni??os")

relation["ACCION"] = relation.pop("accion")
relation["BOTES"] = relation.pop("botes")
relation["FUTURO"] = relation.pop("futuro")
relation["MONTANA"] = relation.pop("monta??a")
relation["MUSICAL"] = relation.pop("musical")
relation["NINOS"] = relation.pop("ni??os")
#-----------------------------------------------------------------------------
print "Categorias: ", categoriesDict
print " "
print "Actividades: ", activitiesDict
print " "
#-----------------------------------------------------------------------------
description = description.mapPartitions(lambda x: hashNames(x, 0))
description = description.mapPartitions(lambda x: hashNames(x, 1))
description = description.map(lambda x: [x[0], x[1], int(x[2]), int(x[3])])
print "Description: "
description.collect()
#-----------------------------------------------------------------------------

Categorias:  {'ACCION': 4, 'BOTES': 5, 'MONTANA': 2, 'NINOS': 3, 'FUTURO': 1, 'MUSICAL': 0}
 
Actividades:  {'"""its a small world"""': 18, 'Enchanted Tales with Belle': 0, 'Astro Orbiter': 1, 'Haunted Mansion': 2, 'Space Mountain': 3, 'Seven Dwarfs Mine Train ': 4, 'Under the Sea - Journey of The Little Mermaid': 5, 'The Barnstormer': 6, 'Monsters, Inc. Laugh Floor': 7, 'Big Thunder Mountain Railroad': 8, 'The Many Adventures of Winnie the Pooh': 9, 'Splash Mountain': 10, 'The Magic Carpets of Aladdin': 11, 'Prince Charming Regal Carrousel': 12, 'Buzz Lightyears Space Ranger Spin': 13, 'Tomorrowland Speedway': 14, 'Mickeys PhilharMagic': 15, 'Mad Tea Party': 21, 'Pirates of the Caribbean': 17, 'Dumbo the Flying Elephant': 19, 'Peter Pans Flight': 20, 'Jungle Cruise': 16}
 
Description: 


[[0, 0, 3, 0],
 [1, 1, 4, 0],
 [2, 1, 5, 1],
 [3, 2, 10, 1],
 [4, 2, 7, 1],
 [5, 3, 2, 1],
 [6, 3, 3, 0],
 [7, 3, 4, 0],
 [8, 2, 6, 1],
 [9, 3, 1, 0],
 [10, 2, 9, 0],
 [11, 3, 2, 0],
 [12, 3, 0, 0],
 [13, 4, 6, 1],
 [14, 1, 2, 1],
 [15, 0, 4, 0],
 [16, 5, 2, 0],
 [17, 5, 6, 1],
 [18, 3, 1, 1],
 [19, 3, 0, 0],
 [20, 3, 1, 0],
 [21, 3, 2, 1]]

# Agrego la comida

In [539]:
def copyList(aList, size):
    vec = copy.copy(aList)
    for i in range(0, size):
        vec[i] = copy.copy(aList[i])
    return vec
def DescriptionToDict(description):
    dicc = {}
    for row in description:
        dicc[row[0]] = row[1:len(row)]
    return dicc

In [540]:
waitTimeFood = 40
SizeTookTimes = len(tookTimes)
size = len(activitiesDict.keys())
activitiesDict["food"] = size
categoriesDict["food"] = len(categoriesDict.keys())
vec = copyList(disney[0:SizeTookTimes], SizeTookTimes)
for i in range(0, SizeTookTimes):
    vec[i][0] = activitiesDict["food"]
    vec[i][1] = waitTimeFood
disney = disney+vec
description = description.collect()
description.append([activitiesDict["food"], categoriesDict["food"], 10, 1])
descriptionDict = DescriptionToDict(description)
description = sc.parallelize(description)
sizeDisney = len(disney)
for i in range(0, sizeDisney):
    activity = disney[i][0]
    disney[i].append(descriptionDict[activity][0])
    disney[i].append(descriptionDict[activity][1])
    disney[i].append(descriptionDict[activity][2])
posDisneyDicc = {
    "activity": 0,
    "waittime": 1,
    "tooktime": 2,
    "category": 3,
    "points": 4,
    "imprescindible": 5
}
print "disney"
disney

disney


[[0, 600, 0, 0, 3, 0],
 [0, 600, 2691, 0, 3, 0],
 [0, 900, 5391, 0, 3, 0],
 [0, 1200, 8091, 0, 3, 0],
 [0, 2100, 10792, 0, 3, 0],
 [0, 2100, 13492, 0, 3, 0],
 [0, 1800, 16192, 0, 3, 0],
 [0, 1800, 18892, 0, 3, 0],
 [0, 2400, 21592, 0, 3, 0],
 [0, 2700, 24292, 0, 3, 0],
 [0, 1800, 26993, 0, 3, 0],
 [0, 1800, 29693, 0, 3, 0],
 [0, 2100, 32393, 0, 3, 0],
 [0, 1500, 35093, 0, 3, 0],
 [0, 1500, 37793, 0, 3, 0],
 [0, 1200, 40493, 0, 3, 0],
 [0, 600, 43194, 0, 3, 0],
 [1, 1200, 0, 1, 4, 0],
 [1, 1800, 2691, 1, 4, 0],
 [1, 2700, 5391, 1, 4, 0],
 [1, 2400, 8091, 1, 4, 0],
 [1, 2400, 10792, 1, 4, 0],
 [1, 1800, 13492, 1, 4, 0],
 [1, 1200, 16192, 1, 4, 0],
 [1, 3000, 18892, 1, 4, 0],
 [1, 1800, 21592, 1, 4, 0],
 [1, 3300, 24292, 1, 4, 0],
 [1, 3000, 26993, 1, 4, 0],
 [1, 1200, 29693, 1, 4, 0],
 [1, 1200, 32393, 1, 4, 0],
 [1, 2400, 35093, 1, 4, 0],
 [1, 2400, 37793, 1, 4, 0],
 [1, 1800, 40493, 1, 4, 0],
 [1, 1200, 43194, 1, 4, 0],
 [2, 900, 0, 1, 5, 1],
 [2, 1200, 2691, 1, 5, 1],
 [2, 1800, 5391,

# Heuristica

In [541]:
MAX_ACTIVITIES = len(activitiesDict.values())
MAX_IMPRESCINDIBLES = len(description.map(lambda x: [x[3], 1]).filter(lambda x: x[0] == 1).collect())

In [542]:
description = description.collect()

In [550]:
def sortByPos(disney, pos):
    return disney.sortBy(lambda x: -x[pos]).collect()
def sortByPosList(rdd, posList, FL):
    allPos = range(0, len(rdd.collect()[0]))
    rdd = rdd.map(lambda x: [[x[i] for i in posList], [x[i] for i in allPos if i not in posList]])
    rdd = rdd.sortByKey(False)
    #rdd = rdd.map(lambda x: x[1]+x[0]).map(lambda x: [x[i] for i in FL])
    return rdd.collect()

In [552]:
disney

[[0, 600, 0, 0, 3, 0],
 [0, 600, 2691, 0, 3, 0],
 [0, 900, 5391, 0, 3, 0],
 [0, 1200, 8091, 0, 3, 0],
 [0, 2100, 10792, 0, 3, 0],
 [0, 2100, 13492, 0, 3, 0],
 [0, 1800, 16192, 0, 3, 0],
 [0, 1800, 18892, 0, 3, 0],
 [0, 2400, 21592, 0, 3, 0],
 [0, 2700, 24292, 0, 3, 0],
 [0, 1800, 26993, 0, 3, 0],
 [0, 1800, 29693, 0, 3, 0],
 [0, 2100, 32393, 0, 3, 0],
 [0, 1500, 35093, 0, 3, 0],
 [0, 1500, 37793, 0, 3, 0],
 [0, 1200, 40493, 0, 3, 0],
 [0, 600, 43194, 0, 3, 0],
 [1, 1200, 0, 1, 4, 0],
 [1, 1800, 2691, 1, 4, 0],
 [1, 2700, 5391, 1, 4, 0],
 [1, 2400, 8091, 1, 4, 0],
 [1, 2400, 10792, 1, 4, 0],
 [1, 1800, 13492, 1, 4, 0],
 [1, 1200, 16192, 1, 4, 0],
 [1, 3000, 18892, 1, 4, 0],
 [1, 1800, 21592, 1, 4, 0],
 [1, 3300, 24292, 1, 4, 0],
 [1, 3000, 26993, 1, 4, 0],
 [1, 1200, 29693, 1, 4, 0],
 [1, 1200, 32393, 1, 4, 0],
 [1, 2400, 35093, 1, 4, 0],
 [1, 2400, 37793, 1, 4, 0],
 [1, 1800, 40493, 1, 4, 0],
 [1, 1200, 43194, 1, 4, 0],
 [2, 900, 0, 1, 5, 1],
 [2, 1200, 2691, 1, 5, 1],
 [2, 1800, 5391,

In [551]:
sortByPosList(sc.parallelize(disney), [5, 4, 2], [0, 1, 5, 2, 4, 3])

[([1, 10, 43194], [3, 600, 2]),
 ([1, 10, 43194], [22, 40, 6]),
 ([1, 10, 40493], [3, 1800, 2]),
 ([1, 10, 40493], [22, 40, 6]),
 ([1, 10, 37793], [3, 1800, 2]),
 ([1, 10, 37793], [22, 40, 6]),
 ([1, 10, 35093], [3, 3000, 2]),
 ([1, 10, 35093], [22, 40, 6]),
 ([1, 10, 32393], [3, 2100, 2]),
 ([1, 10, 32393], [22, 40, 6]),
 ([1, 10, 29693], [3, 3000, 2]),
 ([1, 10, 29693], [22, 40, 6]),
 ([1, 10, 26993], [3, 2700, 2]),
 ([1, 10, 26993], [22, 40, 6]),
 ([1, 10, 24292], [3, 3000, 2]),
 ([1, 10, 24292], [22, 40, 6]),
 ([1, 10, 21592], [3, 2400, 2]),
 ([1, 10, 21592], [22, 40, 6]),
 ([1, 10, 18892], [3, 3900, 2]),
 ([1, 10, 18892], [22, 40, 6]),
 ([1, 10, 16192], [3, 2400, 2]),
 ([1, 10, 16192], [22, 40, 6]),
 ([1, 10, 13492], [3, 3900, 2]),
 ([1, 10, 13492], [22, 40, 6]),
 ([1, 10, 10792], [3, 2700, 2]),
 ([1, 10, 10792], [22, 40, 6]),
 ([1, 10, 8091], [3, 2400, 2]),
 ([1, 10, 8091], [22, 40, 6]),
 ([1, 10, 5391], [3, 1800, 2]),
 ([1, 10, 5391], [22, 40, 6]),
 ([1, 10, 2691], [3, 600, 2]),

In [ ]:
disney = sortByPosList(sc.parallelize(disney), [5, 4, 0], [5, 0, 1, 2, 4, 3])
description = sortByPosList(sc.parallelize(description), [3, 2], [0, 1, 3, 2])

In [495]:
def thereIsOverlap(activity, madeActivities, posDisneyDicc):
    size = len(madeActivities)
    for i in range(0, size):
        I_a = activity[posDisneyDicc["tooktime"]]
        W_a = activity[posDisneyDicc["waittime"]]
        F_a = I_a + W_a
        I_b = madeActivities[i][posDisneyDicc["tooktime"]]
        if F_a > I_b and I_a <= I_b:
            return True
    return False

In [496]:
imprescindibles = 0
activitiesQuant = {}
madeActivities = []
for i in range(0, MAX_ACTIVITIES):
    activitiesQuant[activitiesDict.values()[i]] = 0
print "diccionario de cantidad de vecen que hice la actividad i: "
print activitiesQuant

diccionario de cantidad de vecen que hice la actividad i: 
{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0}


In [517]:
activitiesList = copy.copy(disney)
iteration = 0
while (imprescindibles < MAX_IMPRESCINDIBLES and activitiesList):
    iteration += 1
    activity = activitiesList.pop(0)
    if not thereIsOverlap(activity, madeActivities, posDisneyDicc):
        madeActivities.append(activity)
        activityHash = activity[posDisneyDicc["activity"]]
        activitiesQuant[activityHash] += 1
        if activity[posDisneyDicc["imprescindible"]] and activitiesQuant[activityHash] == 1:
            imprescindibles += 1
print "iteration: ", iteration, " - Size Disney: ", len(disney)
print "Cantidad de imprescindibles", imprescindibles

iteration:  391  - Size Disney:  391
Cantidad de imprescindibles 2
